<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkCrDeltaTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkCrDeltaTable") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
spark

In [3]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

Method 1

In [4]:
schema = [
    StructField("id", IntegerType(), True), # Assuming 'id' is an integer, change if needed
    StructField("name", StringType(), True),
    StructField("salary", IntegerType(), True) # Assuming 'salary' is an integer, change if needed
]
DeltaTable.create(spark)\
          .tableName('employe_table')\
          .addColumns(schema)\
          .location('/content/spark-warehouse')\
          .execute()

In [5]:
spark.sql('select * from employe_table').show()

+---+----+------+
| id|name|salary|
+---+----+------+
+---+----+------+



Method 2

In [6]:
DeltaTable.createIfNotExists(spark)\
          .tableName('employe_table')\
          .addColumns(schema)\
          .location('/content/spark-warehouse')\
          .execute()

Method3

In [7]:
DeltaTable.createOrReplace(spark)\
          .tableName('employe_table')\
          .addColumns(schema)\
          .location('/content/spark-warehouse')\
          .execute()

In [8]:
!cat /content/spark-warehouse/_delta_log/00000000000000000000.json

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"e5a45aef-4c89-4f90-bd97-84a2479018e1","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"salary\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1750059998253}}
{"commitInfo":{"timestamp":1750059998387,"operation":"CREATE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{},"engineInfo":"Apache-Spark/3.2.4 Delta-Lake/2.0.0","txnId":"0fabdacf-53c5-4c18-98b9-b8354a1f7378"}}


In [9]:
!cat /content/spark-warehouse/_delta_log/00000000000000000001.json

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"1eec81c6-b2f6-40a5-a641-3cda4edb647e","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"salary\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1750060171699}}
{"commitInfo":{"timestamp":1750060173205,"operation":"CREATE OR REPLACE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{},"engineInfo":"Apache-Spark/3.2.4 Delta-Lake/2.0.0","txnId":"32cc45d5-7063-4bac-8173-6d5c8c1fdf70"}}


####SQL APPROACH

As in Spark way there are three ways for sql as well, create table, create or replace table and create table if not exists

In [11]:
spark.sql("Create table employeee_table(id int,name string,salary int)using DELTA")

DataFrame[]

In [13]:
spark.sql("select * from employeee_table").show()

+---+----+------+
| id|name|salary|
+---+----+------+
+---+----+------+



In [14]:
!cat /content/spark-warehouse/employee_table/_delta_log/00000000000000000000.json

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"83804a8d-0fd0-448d-800c-5be1734c9963","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"salary\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1750060459580}}
{"commitInfo":{"timestamp":1750060459604,"operation":"CREATE TABLE","operationParameters":{"isManaged":"true","description":null,"partitionBy":"[]","properties":"{}"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{},"engineInfo":"Apache-Spark/3.2.4 Delta-Lake/2.0.0","txnId":"61774687-9690-44da-9b12-2f6a123dfc6b"}}


###  creating delta table Using Dataframe

In [18]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [19]:
df.show()

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|      Wired Earbuds|Electronics Hub|   PROD101| STORE01| 25.99|California|    USA|
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|  Bluetooth Speaker|Electronics Hub|   PROD103| STORE01| 79.50|California|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
|             Webcam|Electronics Hub|   PROD108| STORE01| 55.00|      NULL|    USA|
|     Gaming Headset|      PC Palace|   PROD109| STORE04| 90.00|  New York| 

In [20]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [21]:
spark.sql('select * from Products_table').show()

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|      Wired Earbuds|Electronics Hub|   PROD101| STORE01| 25.99|California|    USA|
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|  Bluetooth Speaker|Electronics Hub|   PROD103| STORE01| 79.50|California|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
|             Webcam|Electronics Hub|   PROD108| STORE01| 55.00|      NULL|    USA|
|     Gaming Headset|      PC Palace|   PROD109| STORE04| 90.00|  New York| 

**END OF CODE**